# Trains and Tests PLDA

In [115]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
sys.path.insert(0, "/home/sjoshi/codes/python/BeatPD/code")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error

#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

# Confusion matrix
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier


from sklearn.pipeline import Pipeline

In [116]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 exp	 newpldatest.ipynb  run3ax.sh	run.sh	 utils
cmd.sh~  exp3ax  path.sh	    run3ax.sh~	run.sh~
conf	 local	 pca_pldapy.ipynb   runFor.sh	sid
data	 mfcc	 README.txt	    runFor.sh~	steps
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 exp	 newpldatest.ipynb  run3ax.sh	run.sh	 utils
cmd.sh~  exp3ax  path.sh	    run3ax.sh~	run.sh~
conf	 local	 pca_pldapy.ipynb   runFor.sh	sid
data	 mfcc	 README.txt	    runFor.sh~	steps


In [117]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
from pca_knn_bpd import pca_knn_bpd

In [120]:
from pca_plda_bpd import pca_plda_bpd

In [125]:
pca_knn_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents, 3)

----- 1004----- 
Training accuracy:  0.3968253968253968
Testing accuracy:  0.3157894736842105
----- 1006----- 
Training accuracy:  0.7142857142857143
Testing accuracy:  0.6666666666666666
----- 1007----- 
Training accuracy:  0.54337899543379
Testing accuracy:  0.543859649122807
----- 1019----- 
Training accuracy:  0.5714285714285714
Testing accuracy:  0.5
----- 1020----- 
Training accuracy:  0.8181818181818182
Testing accuracy:  0.7804878048780488
----- 1023----- 
Training accuracy:  0.4024390243902439
Testing accuracy:  0.3333333333333333
----- 1032----- 
Training accuracy:  0.5323741007194245
Testing accuracy:  0.34210526315789475
----- 1034----- 
Training accuracy:  0.4
Testing accuracy:  0.2
----- 1038----- 
Training accuracy:  0.7195121951219512
Testing accuracy:  0.6744186046511628
----- 1039----- 
Training accuracy:  0.5346534653465347
Testing accuracy:  0.3448275862068966
----- 1043----- 
Training accuracy:  0.52
Testing accuracy:  0.1111111111111111
----- 1044----- 
Training a

In [129]:
pca_plda_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents)

----- 1004----- 
Previous shapes:
(63, 50)
(63,)
Shapes after removing classes:
(63, 50)
(63,)
Accuracy: 0.15789473684210525
----- 1006----- 
Previous shapes:
(28, 50)
(28,)
Shapes after removing classes:
(28, 50)
(28,)
Accuracy: 0.6666666666666666
----- 1007----- 
Previous shapes:
(219, 50)
(219,)
Shapes after removing classes:
(219, 50)
(219,)
Accuracy: 0.5263157894736842
----- 1019----- 
Previous shapes:
(35, 50)
(35,)
Shapes after removing classes:
(35, 50)
(35,)
Accuracy: 0.2
----- 1020----- 
Previous shapes:
(154, 50)
(154,)
Shapes after removing classes:
(154, 50)
(154,)
Accuracy: 0.7804878048780488
----- 1023----- 
Previous shapes:
(82, 50)
(82,)
Shapes after removing classes:
(82, 50)
(82,)
Accuracy: 0.3333333333333333
----- 1032----- 
Previous shapes:
(139, 50)
(139,)
Shapes after removing classes:
(139, 50)
(139,)
Accuracy: 0.2631578947368421
----- 1034----- 
Previous shapes:
(30, 50)
(30,)
Shapes after removing classes:
(30, 50)
(30,)
Accuracy: 0.4
----- 1038----- 
Previous

In [123]:
def final_score(mse_per_subjectid, nb_files_per_subject_id):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
    print('numerator : ', numerator)
    print('denominator : ', denominator)
    print('Final score : ', np.divide(numerator, denominator))
    
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

numerator :  2.9600025680608737
denominator :  136.91737735752739
Final score :  0.021618896192639788
numerator :  6.704932590433186
denominator :  72.47516774327755
Final score :  0.09251351599741697


In [130]:
print(train_nb_files_per_subjectid)
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

[63, 28, 219, 35, 154, 82, 139, 30, 164, 101, 25, 57, 71, 64, 154]
numerator :  2.9600025680608737
denominator :  136.91737735752739
Final score :  0.021618896192639788
numerator :  6.704932590433186
denominator :  72.47516774327755
Final score :  0.09251351599741697


In [543]:
def do_confusion_matrix(y_test, predictions):
    print('y test : ', np.unique(y_test))
    LABELS_NEW = np.unique(y_test)
    n_classes=np.unique(y_test)
    print("Confusion Matrix:")
    confusion_matrix = metrics.confusion_matrix(y_test, predictions)
    print(confusion_matrix)
    normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

    print("")
    print("Confusion matrix (normalised to % of total test data):")
    print(normalised_confusion_matrix)

    # Plot Results:
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.imshow(
        normalised_confusion_matrix,
        interpolation='nearest',
        cmap=plt.cm.rainbow
    )
    plt.title("Confusion matrix \n(normalised to % of total test data)")
    plt.colorbar()
    tick_marks = np.arange(n_classes)
    plt.xticks(tick_marks, LABELS_NEW, rotation=90)
    plt.yticks(tick_marks, LABELS_NEW)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()